In [1]:
import pennylane as qml
import numpy as np

T = 30  # Number of time steps
noise_param = 0.75  # Noise parameter
num_qubits = 1
states = []

dev = qml.device("default.mixed", wires=num_qubits)

def random_pure_state(num_qubits):
    """
    Genera uno stato puro casuale.
    """
    state = np.random.normal(size=(2**num_qubits, 2)) + 1j * np.random.normal(size=(2**num_qubits, 2))
    state = state[:, 0] + 1j * state[:, 1]
    return state / np.linalg.norm(state)

# Genera uno stato puro casuale
initial_state_vector = random_pure_state(num_qubits)

# Converti il vettore di stato in una matrice di densità
initial_state = np.outer(initial_state_vector, np.conj(initial_state_vector))

purity = np.trace(np.matmul(initial_state, initial_state))
print(f"Initial Purity: {purity.real:.6f}")

@qml.qnode(dev)
def apply_noise(state, noise_param):
    qml.QubitDensityMatrix(state, wires=range(num_qubits))
    qml.DepolarizingChannel(noise_param, wires=0)
    return qml.density_matrix(wires=range(num_qubits))

for t in range(T + 1):
    rho_t = apply_noise(initial_state, noise_param * t / T)
    states.append(rho_t)
    print(f"Step {t}:")
    print("Shape:", rho_t.shape)
    
    # Calculate and print purity
    purity = np.trace(np.matmul(rho_t, rho_t))
    print(f"Purity: {purity.real:.6f}")
    print()

# Verifica della purezza finale
final_purity = np.trace(np.matmul(states[-1], states[-1]))
print(f"Final Purity: {final_purity.real:.6f}")

Initial Purity: 1.000000
Step 0:
Shape: (2, 2)
Purity: 1.000000

Step 1:
Shape: (2, 2)
Purity: 0.967222

Step 2:
Shape: (2, 2)
Purity: 0.935556

Step 3:
Shape: (2, 2)
Purity: 0.905000

Step 4:
Shape: (2, 2)
Purity: 0.875556

Step 5:
Shape: (2, 2)
Purity: 0.847222

Step 6:
Shape: (2, 2)
Purity: 0.820000

Step 7:
Shape: (2, 2)
Purity: 0.793889

Step 8:
Shape: (2, 2)
Purity: 0.768889

Step 9:
Shape: (2, 2)
Purity: 0.745000

Step 10:
Shape: (2, 2)
Purity: 0.722222

Step 11:
Shape: (2, 2)
Purity: 0.700556

Step 12:
Shape: (2, 2)
Purity: 0.680000

Step 13:
Shape: (2, 2)
Purity: 0.660556

Step 14:
Shape: (2, 2)
Purity: 0.642222

Step 15:
Shape: (2, 2)
Purity: 0.625000

Step 16:
Shape: (2, 2)
Purity: 0.608889

Step 17:
Shape: (2, 2)
Purity: 0.593889

Step 18:
Shape: (2, 2)
Purity: 0.580000

Step 19:
Shape: (2, 2)
Purity: 0.567222

Step 20:
Shape: (2, 2)
Purity: 0.555556

Step 21:
Shape: (2, 2)
Purity: 0.545000

Step 22:
Shape: (2, 2)
Purity: 0.535556

Step 23:
Shape: (2, 2)
Purity: 0.527222

S

In [74]:
import torch
import torch.nn as nn
import torch.optim as optim

class DenoisingNetwork(nn.Module):
    def __init__(self):
        super(DenoisingNetwork, self).__init__()
        self.fc1 = nn.Linear(4, 8) 
        self.fc2 = nn.Linear(8, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.fc4(x)

In [76]:
model = DenoisingNetwork()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    total_loss = 0
    xt = torch.abs(torch.tensor(states[T])).float()
    print(xt.dtype)
    for t in range(T, 0, -1):
        optimizer.zero_grad()
        xt = model(xt.flatten().detach()).reshape(2,2)
        xt_target = torch.tensor(states[t - 1])
        loss =  torch.abs(xt - xt_target).mean()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(total_loss/T)
    purity = torch.trace(torch.matmul(xt, xt))
    print(f"Purity of xt: {purity.real:.6f}")
    purity = torch.trace(torch.matmul(xt_target, xt_target))
    print(f"Purity of xt_target: {purity.real:.6f}")

torch.float32
0.32605439278087606
Purity of xt: 0.399625
Purity of xt_target: 1.000000
torch.float32
0.2686777380575323
Purity of xt: 0.478500
Purity of xt_target: 1.000000
torch.float32
0.2099003460260622
Purity of xt: 0.568319
Purity of xt_target: 1.000000
torch.float32
0.15762916638947536
Purity of xt: 0.666503
Purity of xt_target: 1.000000
torch.float32
0.13102540952200842
Purity of xt: 0.580076
Purity of xt_target: 1.000000
torch.float32
0.13232418311928265
Purity of xt: 0.630889
Purity of xt_target: 1.000000
torch.float32
0.13365165535733528
Purity of xt: 0.595193
Purity of xt_target: 1.000000
torch.float32
0.12997309782679714
Purity of xt: 0.620111
Purity of xt_target: 1.000000
torch.float32
0.1316343139506164
Purity of xt: 0.612510
Purity of xt_target: 1.000000
torch.float32
0.13100497357689125
Purity of xt: 0.606266
Purity of xt_target: 1.000000


In [12]:
dev = qml.device("default.qubit", wires=num_qubits)

class QuantumDenoising(nn.Module):
    def __init__(self, n_layers=10, n_qubits=3, n_qubits1=2, n_a_qubits=1, device='default.qubit.torch'):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_qubits1 = n_qubits1
        self.n_a_qubits = n_a_qubits
        self.n_layers = n_layers
        self.device = device

        self.dev = qml.device(self.device, wires=self.n_qubits)
        self.dev1 = qml.device(self.device, wires=self.n_qubits1)
        
        self.qnode = qml.QNode(self.circuit, self.dev, interface='torch')
        self.qnode1 = qml.QNode(self.circuit1, self.dev1, interface='torch')
        
        self.weight_shapes = {"weights": qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=self.n_qubits)}
        self.weight_shapes1 = {"weights": qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=self.n_qubits1)}
        
        self.qlayer = qml.qnn.TorchLayer(self.qnode, self.weight_shapes)
        self.qlayer1 = qml.qnn.TorchLayer(self.qnode1, self.weight_shapes1)

    def circuit(self, inputs, weights):
        qml.AmplitudeEmbedding(features=inputs, wires=range(self.n_qubits1), normalize=True)
        qml.StronglyEntanglingLayers(weights=weights, wires=range(self.n_qubits))
        return qml.state()
    
    def circuit1(self, inputs, weights):
        qml.AmplitudeEmbedding(features=inputs, wires=range(self.n_qubits1), normalize=True)
        qml.StronglyEntanglingLayers(weights=weights, wires=range(self.n_qubits1))
        return qml.state()
    
    def forward(self, x):
        probs = self.qlayer(x)
        probsgiven0 = probs[: (2 ** (self.n_qubits - self.n_a_qubits))]
        probsgiven0 /= torch.sum(probs)
        return self.qlayer1(probsgiven0)

In [77]:
model = QuantumDenoising(n_layers=10, n_qubits=3, n_qubits1=2, n_a_qubits=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    total_loss = 0
    xt = states[T]
    for t in range(T, 0, -1):
        optimizer.zero_grad()
        xt = model(torch.tensor(xt).flatten()).reshape(2,2)
        xt_target = torch.tensor(states[t - 1])
        loss =  torch.abs(xt - xt_target).mean()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(total_loss/T)
    purity = torch.trace(torch.matmul(xt, xt))
    print(f"Purity of xt: {purity.real:.6f}")
    purity = torch.trace(torch.matmul(xt_target, xt_target))
    print(f"Purity of xt_target: {purity.real:.6f}")

C:\Users\pevel\AppData\Local\Temp\ipykernel_10796\79636556.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xt = model(torch.tensor(xt).flatten()).reshape(2,2)


0.6228681516516714
Purity of xt: -0.689171
Purity of xt_target: 1.000000
0.4325686275028804
Purity of xt: -0.055072
Purity of xt_target: 1.000000
0.2029240423006781
Purity of xt: 0.941784
Purity of xt_target: 1.000000
0.1318559283768778
Purity of xt: 0.972777
Purity of xt_target: 1.000000
0.13239480486878652
Purity of xt: 0.996243
Purity of xt_target: 1.000000
0.13224905282362506
Purity of xt: 0.994065
Purity of xt_target: 1.000000
0.13877149269820807
Purity of xt: 0.997370
Purity of xt_target: 1.000000
0.1362584277562448
Purity of xt: 0.989163
Purity of xt_target: 1.000000
0.12479639340689702
Purity of xt: 0.995750
Purity of xt_target: 1.000000
0.13058117713688097
Purity of xt: 0.997752
Purity of xt_target: 1.000000
